<a href="https://colab.research.google.com/github/aidyai/rss-ai/blob/main/rss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade --quiet  apify-client langchain-openai langchain
!pip install feedparser newspaper3k openai
!pip install openai==0.28


In [3]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.utilities import ApifyWrapper
from langchain_core.documents import Document

import feedparser
import requests
from bs4 import BeautifulSoup
import openai
from newspaper import Article

In [4]:
import os
os.environ["APIFY_API_TOKEN"] = "apify_api_r2J5EYAZ2qOeCTSBDqN3RJ2IraTke911bWjC"
os.environ["OPENAI_API_KEY"] = ""


In [6]:
# RSS feed URL
rss_feed_url = 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen'

# Parse the RSS feed
feed = feedparser.parse(rss_feed_url)

# Get the latest 5 articles
latest_articles = feed.entries[6:7]

In [8]:
for article in latest_articles:
    # Get the title and article URL
    title = article.title
    article_url = article.link
    print(article_url)

https://news.google.com/rss/articles/CBMiV2h0dHBzOi8vdGhld2lsbG5ld3MuY29tL29waW5pb24tY3VsdHVyYWwtc2ltaWxhcml0aWVzLWJldHdlZW4tYWt3YS1pYm9taXRlcy1hbmQtbmRpZ2JvL9IBAA?oc=5


In [7]:
apify = ApifyWrapper()

In [8]:
urls_to_crawl = []
# Loop through the latest articles
for article in latest_articles:
    # Get the title and article URL
    title = article.title
    article_url = article.link
    # Print the article URL
    #print(article_url)
    # Append the article URL to the list of URLs to crawl
    urls_to_crawl.append({"url": article_url})

# Use the list of URLs to call the Apify actor
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": urls_to_crawl},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

In [9]:
t = loader.load()
t

[Document(page_content='EFCC arrests 44 suspected internet fraudsters in Calabar\nBy Soni Daniel, Abuja \nInvestigators at the Uyo Zonal Command of the Economic and Financial Crimes Commission, EFCC, have arrested forty-four(44) suspected internet fraudsters.\nThe Spokesman for the EFCC Dele Oyewale said the suspected fraudsters were arrested in an early morning sting operation on Thursday at Satellite town, Tinapa junction and parliamentary extension areas of Calabar in Cross River State.\nREAD ALSO: EFCC arrests 64 suspected internet fraudsters in Osun\nOyewqle listed the Items recovered from them to include six exotic cars, 16 laptops and 38 mobile phones.\nThe statment said the suspects would be charged to court as soon as investigations are concluded.', metadata={'source': 'https://news.google.com/rss/articles/CBMiXWh0dHBzOi8vd3d3LnZhbmd1YXJkbmdyLmNvbS8yMDI0LzA1L2VmY2MtYXJyZXN0cy00NC1zdXNwZWN0ZWQtaW50ZXJuZXQtZnJhdWRzdGVycy1pbi1jYWxhYmFyL9IBYWh0dHBzOi8vd3d3LnZhbmd1YXJkbmdyLmNvbS8yM

In [16]:
import json
# Extracting content from the list
extracted_content = [document.page_content for document in t]

# Printing the extracted content
print(extracted_content)

['EFCC arrests 44 suspected internet fraudsters in Calabar\nBy Soni Daniel, Abuja \nInvestigators at the Uyo Zonal Command of the Economic and Financial Crimes Commission, EFCC, have arrested forty-four(44) suspected internet fraudsters.\nThe Spokesman for the EFCC Dele Oyewale said the suspected fraudsters were arrested in an early morning sting operation on Thursday at Satellite town, Tinapa junction and parliamentary extension areas of Calabar in Cross River State.\nREAD ALSO: EFCC arrests 64 suspected internet fraudsters in Osun\nOyewqle listed the Items recovered from them to include six exotic cars, 16 laptops and 38 mobile phones.\nThe statment said the suspects would be charged to court as soon as investigations are concluded.']


In [22]:
import openai


In [ ]:
# Send the content to OpenAI to get topic and category tags
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=extracted_content,
    max_tokens=50,  # Adjust as needed
    n=1,  # Number of completions to generate
    stop=["\n"],  # Stop completion at the end of the text
    temperature=0.5,  # Adjust as needed
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    logit_bias={},
    logprobs=0,
    echo=True  # Print the input prompt along with the completion
)

# Extract the generated tags from the response
generated_tags = response.choices[0].text.strip().split("\n")

# Extract 5 topic tags and 5 article category tags
topic_tags = generated_tags[:5]
article_category_tags = generated_tags[5:10]

# Print the extracted tags
print("Topic Tags:", topic_tags)
print("Article Category Tags:", article_category_tags)

In [24]:
def generate_tags(article_text):
    # Prompt for generating topic tags
    topic_prompt = f"Generate 5 topic tags for the given article:\n{article_text}\n---\n"
    # Prompt for generating article category tags
    category_prompt = f"Generate 5 article category tags for the given article:\n{article_text}\n---\n"

    # Call OpenAI's classification model to generate tags
    topic_response = openai.Completion.create(
        model="gpt-3.5-turbo-instruct",
        prompt=topic_prompt,
        max_tokens=50,
        n=5,
        stop=["\n"]
    )

    # Extract tags from the response
    topic_tags = [choice["text"].strip() for choice in topic_response.choices]
    #category_tags = [choice["text"].strip() for choice in category_response.choices]

    return topic_tags  #, category_tags


In [25]:
# Generate tags for the example article text
topic_tags, category_tags = generate_tags(extracted_content)

# Print the generated tags
print("Generated Topic Tags:")
for tag in topic_tags:
    print("-", tag)

#print("\nGenerated Article Category Tags:")
#for tag in category_tags:
#    print("-", tag)


AuthenticationError: Incorrect API key provided: sk-nUdwN***************************************fnpE. You can find your API key at https://platform.openai.com/account/api-keys.

In [36]:
t = loader.load()
t

[Document(page_content='By Chioma Onuegbu Uyo\nGOVERNOR Umo Eno has continued to demonstrate his readiness, determination, and commitment to fulfilling his contract with the people of Akwa Ibom State including creating job opportunities and enlarging the path for economic prosperity.\nAnd the governor at different fora emphasized that for his administration to achieve this contract there was need to revive the entrepreneurial spirit of the people through comprehensive training and support.\nAccordingly, as an ardent advocate of entrepreneurship development, as soon as he came on board, he hit the ground running, by putting in place variety of initiatives, popular among them is a Business School under the auspices of Ibom-Leadership Entrepreneurial Development (Ibom-LED).\nThe initiative which has its Headquarters at the E-Library Complex along IBB Avenue Uyo has Pastor ImaAbasi Jacob as the team lead and Mrs Ekaette Umoh as the Director General.\nAnd according to Ibom-LED team , the in

In [ ]:
t = loader.load()
t

In [27]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x

In [ ]:
!TORCH_CUDA_ARCH_LIST="7.5+PTX" pip install git+https://github.com/casper-hansen/AutoAWQ_kernels.git@main

In [ ]:
!pip install git+https://github.com/casper-hansen/AutoAWQ.git@main

In [ ]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, AwqConfig

In [ ]:
import torch
import awq_ext

In [ ]:
device = "cuda"

# To fit the memory of a standard T4 GPU of a google colab, we'll use the quantized version of mistal 7b instruct
# For faster inference, we can also use module fusing
# quantization_config = AwqConfig(
#     bits=4,
#     fuse_max_seq_len=4096,
#     do_fuse=True,
# )
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.2-AWQ",
#     quantization_config=quantization_config,
).to(device)
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.2-AWQ")
tokenizer.padding_side = "left"

In [ ]:
# Prompt for generating topic tags
topic_prompt = f"Generate 5 topic tags for the given article:\n{article_text}\n---\n"
# Prompt for generating article category tags
category_prompt = f"Generate 5 article category tags for the given article:\n{article_text}\n---\n"


In [ ]:
BATCH_SIZE = 1
NUM_ITER = 2

ALL_IDEAS = []

messages = [{"role": "user", "content": PROMPT},]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
encodeds = encodeds.repeat(BATCH_SIZE, 1)
len_prompt = encodeds.shape[1]
model_inputs = encodeds.to(device)


for _ in tqdm(range(NUM_ITER)):
    generated_ids = model.generate(model_inputs, max_new_tokens=1024, do_sample=True)
    generated_ids = generated_ids[:, len_prompt:]
    decoded = tokenizer.batch_decode(generated_ids)

    for dec_ in decoded:
        try:
            dec = dec_
            dec = dec.replace("", "").replace("\n\n", "\n")
            dec = dec.split("\n")
            dec = [el for el in dec if el[0].isdigit()]
            if len(dec) != 10:
                continue
            dec = [
                el[2:].strip() if el[:2] == f"{idx + 1}." else el[3:].strip() if el[:3] == f"{idx + 1}." else ""
                for idx, el in enumerate(dec)
            ]
            dec = [el for el in dec if el]
            if len(dec) != 10:
                continue
            ALL_IDEAS.extend(dec)
        except Exception:
            print("Skipped")
            pass


In [ ]:
prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [9]:
rss_feed_url = 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen'

# Parse the RSS feed
feed = feedparser.parse(rss_feed_url)

# Get the latest 5 articles
latest_articles = feed.entries[1:5]

In [16]:
for article in latest_articles:
    # Get the title and article URL
    title = article.title
    article_url = article.link

    # Use newspaper3k to extract article text
    article_text = ""
    try:
        news_article = Article(article_url)
        news_article.download()
        news_article.parse()
        article_text = news_article.text
    except Exception as e:
        print("Error fetching article text:", e)

    print("Title:", title)
    print("Article URL:", article_url)
    print("Article Text:", article_text[:600])  # Displaying the first 150 characters of the article text
    print("-" * 250)

Error fetching article text: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.vanguardngr.com/2024/05/how-eno-stimulating-akwa-ibom-economy-through-entrepreneurship-development/ on URL https://news.google.com/rss/articles/CBMib2h0dHBzOi8vd3d3LnZhbmd1YXJkbmdyLmNvbS8yMDI0LzA1L2hvdy1lbm8tc3RpbXVsYXRpbmctYWt3YS1pYm9tLWVjb25vbXktdGhyb3VnaC1lbnRyZXByZW5ldXJzaGlwLWRldmVsb3BtZW50L9IBc2h0dHBzOi8vd3d3LnZhbmd1YXJkbmdyLmNvbS8yMDI0LzA1L2hvdy1lbm8tc3RpbXVsYXRpbmctYWt3YS1pYm9tLWVjb25vbXktdGhyb3VnaC1lbnRyZXByZW5ldXJzaGlwLWRldmVsb3BtZW50L2FtcC8?oc=5
Title: How Eno stimulating Akwa Ibom economy through entrepreneurship development - Vanguard
Article URL: https://news.google.com/rss/articles/CBMib2h0dHBzOi8vd3d3LnZhbmd1YXJkbmdyLmNvbS8yMDI0LzA1L2hvdy1lbm8tc3RpbXVsYXRpbmctYWt3YS1pYm9tLWVjb25vbXktdGhyb3VnaC1lbnRyZXByZW5ldXJzaGlwLWRldmVsb3BtZW50L9IBc2h0dHBzOi8vd3d3LnZhbmd1YXJkbmdyLmNvbS8yMDI0LzA1L2hvdy1lbm8tc3RpbXVsYXRpbmctYWt3YS1pYm9tLWVjb25vbXktdGhyb3VnaC1lbnRyZXByZW5ldXJza

In [ ]:
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

# Load HTML
#rss_feed_url = 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen'
#loader = AsyncChromiumLoader(["https://www.wsj.com"])
loader = AsyncChromiumLoader(["https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen"])
html = loader.load()

In [ ]:
!pip install feedparser newspaper3k openai

In [2]:
import feedparser
import requests
from bs4 import BeautifulSoup
import openai
from newspaper import Article

In [3]:
# RSS feed URL
rss_feed_url = 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen'

# Parse the RSS feed
feed = feedparser.parse(rss_feed_url)

# Get the latest 5 articles
latest_articles = feed.entries[:1]

In [ ]:
feed

In [4]:
latest_articles

[{'title': 'How Eno stimulating Akwa Ibom economy through entrepreneurship development - Vanguard',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen',
   'value': 'How Eno stimulating Akwa Ibom economy through entrepreneurship development - Vanguard'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://news.google.com/rss/articles/CBMib2h0dHBzOi8vd3d3LnZhbmd1YXJkbmdyLmNvbS8yMDI0LzA1L2hvdy1lbm8tc3RpbXVsYXRpbmctYWt3YS1pYm9tLWVjb25vbXktdGhyb3VnaC1lbnRyZXByZW5ldXJzaGlwLWRldmVsb3BtZW50L9IBc2h0dHBzOi8vd3d3LnZhbmd1YXJkbmdyLmNvbS8yMDI0LzA1L2hvdy1lbm8tc3RpbXVsYXRpbmctYWt3YS1pYm9tLWVjb25vbXktdGhyb3VnaC1lbnRyZXByZW5ldXJzaGlwLWRldmVsb3BtZW50L2FtcC8?oc=5'}],
  'link': 'https://news.google.com/rss/articles/CBMib2h0dHBzOi8vd3d3LnZhbmd1YXJkbmdyLmNvbS8yMDI0LzA1L2hvdy1lbm8tc3RpbXVsYXRpbmctYWt3YS1pYm9tLWVjb25vbXktdGhyb3VnaC1lbnRyZXByZW5ldXJzaGlwLWRldmVsb3BtZW50L9IBc2h0dHBzOi8v